In [3]:
from pyspark.sql import SparkSession

In [5]:
# Initialize SparkSession with Hive support
spark = SparkSession.builder \
    .appName("Pipeline Code") \
    .config("hdfs://localhost:9000/user/hive/warehouse", "/usr/local/spark/jars/mysql-connector-java-8.0.30.jar") \
    .enableHiveSupport() \
    .getOrCreate()


24/07/08 01:28:15 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/07/08 01:28:15 INFO SharedState: Warehouse path is 'file:/home/azureuser/jupyter-env/notebooks/spark-warehouse'.


In [6]:
csv_file_path = "hdfs://localhost:9000//user/input/ClimateData/Weather_casualty_dataset_with_year.csv"

In [7]:
climate_table = "climate_data"
properties = {
    "user": "root",
    "password": "AzurePassword",
    "driver": "com.mysql.cj.jdbc.Driver"
}

In [8]:

df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

24/07/08 01:29:04 INFO InMemoryFileIndex: It took 2 ms to list leaf files for 1 paths.
24/07/08 01:29:04 INFO InMemoryFileIndex: It took 2 ms to list leaf files for 1 paths.
24/07/08 01:29:04 INFO FileSourceStrategy: Pushed Filters: 
24/07/08 01:29:04 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#816, None)) > 0)
24/07/08 01:29:05 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 353.8 KiB, free 366.0 MiB)
24/07/08 01:29:05 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 34.6 KiB, free 365.9 MiB)
24/07/08 01:29:05 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on myvm.internal.cloudapp.net:39661 (size: 34.6 KiB, free: 366.3 MiB)
24/07/08 01:29:05 INFO SparkContext: Created broadcast 0 from csv at NativeMethodAccessorImpl.java:0
24/07/08 01:29:05 INFO FileSourceScanExec: Planning scan with bin packing, max size: 15685023 bytes, open cost is considered as scanning 4194304 bytes.
24/07/08 01:29:0

In [9]:
df.printSchema()


root
 |-- BEGIN_YEARMONTH: integer (nullable = true)
 |-- BEGIN_DAY: integer (nullable = true)
 |-- BEGIN_TIME: integer (nullable = true)
 |-- END_YEARMONTH: integer (nullable = true)
 |-- END_DAY: integer (nullable = true)
 |-- END_TIME: integer (nullable = true)
 |-- EPISODE_ID: integer (nullable = true)
 |-- EVENT_ID: integer (nullable = true)
 |-- STATE: string (nullable = true)
 |-- STATE_FIPS: integer (nullable = true)
 |-- YEAR10: integer (nullable = true)
 |-- MONTH_NAME: string (nullable = true)
 |-- EVENT_TYPE: string (nullable = true)
 |-- CZ_TYPE: string (nullable = true)
 |-- CZ_FIPS: integer (nullable = true)
 |-- CZ_NAME: string (nullable = true)
 |-- WFO: string (nullable = true)
 |-- BEGIN_DATE_TIME: string (nullable = true)
 |-- CZ_TIMEZONE: string (nullable = true)
 |-- END_DATE_TIME: string (nullable = true)
 |-- INJURIES_DIRECT: integer (nullable = true)
 |-- INJURIES_INDIRECT: integer (nullable = true)
 |-- DEATHS_DIRECT: integer (nullable = true)
 |-- DEATHS_INDI

In [10]:
# Show the first few rows (optional)
df.show()

+---------------+---------+----------+-------------+-------+--------+----------+--------+--------------+----------+------+----------+-----------------+-------+-------+--------------------+---+----------------+-----------+----------------+---------------+-----------------+-------------+---------------+---------------+------------+-----------------+---------+--------------+-----------+--------+-----------+----------+---------+-------------+------------------+-----------------+-----------------+-----------+-------------+--------------+---------+-----------+-------------+---------+---------+-------+---------+--------------------+--------------------+-----------+----------------+------+
|BEGIN_YEARMONTH|BEGIN_DAY|BEGIN_TIME|END_YEARMONTH|END_DAY|END_TIME|EPISODE_ID|EVENT_ID|         STATE|STATE_FIPS|YEAR10|MONTH_NAME|       EVENT_TYPE|CZ_TYPE|CZ_FIPS|             CZ_NAME|WFO| BEGIN_DATE_TIME|CZ_TIMEZONE|   END_DATE_TIME|INJURIES_DIRECT|INJURIES_INDIRECT|DEATHS_DIRECT|DEATHS_INDIRECT|DAMAGE_

24/07/08 01:29:43 INFO FileSourceStrategy: Pushed Filters: 
24/07/08 01:29:43 INFO FileSourceStrategy: Post-Scan Filters: 
24/07/08 01:29:43 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 353.7 KiB, free 365.5 MiB)
24/07/08 01:29:43 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 34.5 KiB, free 365.5 MiB)
24/07/08 01:29:43 INFO BlockManagerInfo: Added broadcast_4_piece0 in memory on myvm.internal.cloudapp.net:39661 (size: 34.5 KiB, free: 366.2 MiB)
24/07/08 01:29:43 INFO SparkContext: Created broadcast 4 from showString at NativeMethodAccessorImpl.java:0
24/07/08 01:29:43 INFO FileSourceScanExec: Planning scan with bin packing, max size: 15685023 bytes, open cost is considered as scanning 4194304 bytes.
24/07/08 01:29:43 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/07/08 01:29:43 INFO DAGScheduler: Got job 2 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
2

In [11]:
# Define the Hive database and table name
hive_db = 'climatedata'
hive_table = 'climate_data'

In [12]:
# Create the database if it does not exist (optional)
spark.sql(f"CREATE DATABASE IF NOT EXISTS {hive_db}")

24/07/08 01:30:21 INFO HiveUtils: Initializing HiveMetastoreConnection version 2.3.9 using Spark classes.
24/07/08 01:30:21 INFO HiveClientImpl: Warehouse location for Hive client (version 2.3.9) is file:/home/azureuser/jupyter-env/notebooks/spark-warehouse
24/07/08 01:30:21 INFO metastore: Mestastore configuration hive.metastore.filter.hook changed from org.apache.hadoop.hive.ql.security.authorization.plugin.AuthorizationMetaStoreFilterHook to org.apache.hadoop.hive.metastore.DefaultMetaStoreFilterHookImpl
24/07/08 01:30:21 INFO HiveMetaStore: 0: Cleaning up thread local RawStore...
24/07/08 01:30:21 INFO audit: ugi=azureuser	ip=unknown-ip-addr	cmd=Cleaning up thread local RawStore...	
24/07/08 01:30:21 INFO HiveMetaStore: 0: Done cleaning up thread local RawStore
24/07/08 01:30:21 INFO audit: ugi=azureuser	ip=unknown-ip-addr	cmd=Done cleaning up thread local RawStore	
24/07/08 01:30:21 INFO HiveMetaStore: 0: get_database: default
24/07/08 01:30:21 INFO audit: ugi=azureuser	ip=unknown

DataFrame[]

In [13]:
spark.sql(f"USE {hive_db}")


24/07/08 01:30:47 INFO HiveMetaStore: 0: get_database: global_temp
24/07/08 01:30:47 INFO audit: ugi=azureuser	ip=unknown-ip-addr	cmd=get_database: global_temp	
24/07/08 01:30:47 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
24/07/08 01:30:47 INFO HiveMetaStore: 0: get_database: climatedata
24/07/08 01:30:47 INFO audit: ugi=azureuser	ip=unknown-ip-addr	cmd=get_database: climatedata	


DataFrame[]

In [14]:
df.write.mode('overwrite').saveAsTable(f"{hive_db}.{hive_table}")


24/07/08 01:31:07 INFO HiveMetaStore: 0: get_table : db=climatedata tbl=climate_data
24/07/08 01:31:07 INFO audit: ugi=azureuser	ip=unknown-ip-addr	cmd=get_table : db=climatedata tbl=climate_data	
24/07/08 01:31:07 INFO HiveMetaStore: 0: get_database: climatedata
24/07/08 01:31:07 INFO audit: ugi=azureuser	ip=unknown-ip-addr	cmd=get_database: climatedata	
24/07/08 01:31:07 INFO HiveMetaStore: 0: get_table : db=climatedata tbl=climate_data
24/07/08 01:31:07 INFO audit: ugi=azureuser	ip=unknown-ip-addr	cmd=get_table : db=climatedata tbl=climate_data	
24/07/08 01:31:07 INFO HiveMetaStore: 0: get_table : db=climatedata tbl=climate_data
24/07/08 01:31:07 INFO audit: ugi=azureuser	ip=unknown-ip-addr	cmd=get_table : db=climatedata tbl=climate_data	
24/07/08 01:31:07 INFO InMemoryFileIndex: It took 3 ms to list leaf files for 1 paths.
24/07/08 01:31:07 INFO HiveMetaStore: 0: get_database: climatedata
24/07/08 01:31:07 INFO audit: ugi=azureuser	ip=unknown-ip-addr	cmd=get_database: climatedata	


In [15]:
spark.sql(f"SELECT * FROM {hive_db}.{hive_table}").show()

24/07/08 01:31:52 INFO HiveMetaStore: 0: get_database: climatedata
24/07/08 01:31:52 INFO audit: ugi=azureuser	ip=unknown-ip-addr	cmd=get_database: climatedata	
24/07/08 01:31:52 INFO HiveMetaStore: 0: get_table : db=climatedata tbl=climate_data
24/07/08 01:31:52 INFO audit: ugi=azureuser	ip=unknown-ip-addr	cmd=get_table : db=climatedata tbl=climate_data	
24/07/08 01:31:52 INFO HiveMetaStore: 0: get_table : db=climatedata tbl=climate_data
24/07/08 01:31:52 INFO audit: ugi=azureuser	ip=unknown-ip-addr	cmd=get_table : db=climatedata tbl=climate_data	
24/07/08 01:31:52 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
24/07/08 01:31:52 INFO FileSourceStrategy: Pushed Filters: 
24/07/08 01:31:52 INFO FileSourceStrategy: Post-Scan Filters: 
24/07/08 01:31:52 INFO MemoryStore: Block broadcast_8 stored as values in memory (estimated size 373.9 KiB, free 364.8 MiB)
24/07/08 01:31:52 INFO MemoryStore: Block broadcast_8_piece0 stored as bytes in memory (estimated size 36.8 KiB

+---------------+---------+----------+-------------+-------+--------+----------+--------+--------------+----------+------+----------+--------------------+-------+-------+--------------------+---+----------------+-----------+----------------+---------------+-----------------+-------------+---------------+---------------+------------+--------------------+---------+--------------+-----------+--------+-----------+----------+---------+-------------+------------------+-----------------+-----------------+-----------+-------------+--------------------+---------+-----------+--------------------+---------+---------+-------+-------+--------------------+--------------------+-----------+----------------+------+
|BEGIN_YEARMONTH|BEGIN_DAY|BEGIN_TIME|END_YEARMONTH|END_DAY|END_TIME|EPISODE_ID|EVENT_ID|         STATE|STATE_FIPS|YEAR10|MONTH_NAME|          EVENT_TYPE|CZ_TYPE|CZ_FIPS|             CZ_NAME|WFO| BEGIN_DATE_TIME|CZ_TIMEZONE|   END_DATE_TIME|INJURIES_DIRECT|INJURIES_INDIRECT|DEATHS_DIRECT|DEA

24/07/08 01:31:53 INFO Executor: Finished task 0.0 in stage 4.0 (TID 10). 10609 bytes result sent to driver
24/07/08 01:31:53 INFO TaskSetManager: Finished task 0.0 in stage 4.0 (TID 10) in 93 ms on myvm.internal.cloudapp.net (executor driver) (1/1)
24/07/08 01:31:53 INFO TaskSchedulerImpl: Removed TaskSet 4.0, whose tasks have all completed, from pool 
24/07/08 01:31:53 INFO DAGScheduler: ResultStage 4 (showString at NativeMethodAccessorImpl.java:0) finished in 0.101 s
24/07/08 01:31:53 INFO DAGScheduler: Job 4 is finished. Cancelling potential speculative or zombie tasks for this job
24/07/08 01:31:53 INFO TaskSchedulerImpl: Killing all running tasks in stage 4: Stage finished
24/07/08 01:31:53 INFO DAGScheduler: Job 4 finished: showString at NativeMethodAccessorImpl.java:0, took 0.103404 s


In [16]:
databases = spark.sql("SHOW DATABASES")
databases.show()

+-----------+
|  namespace|
+-----------+
|climatedata|
|    default|
+-----------+



24/07/08 01:36:43 INFO HiveMetaStore: 0: get_databases: *
24/07/08 01:36:43 INFO audit: ugi=azureuser	ip=unknown-ip-addr	cmd=get_databases: *	
24/07/08 01:36:43 INFO CodeGenerator: Code generated in 43.221488 ms
24/07/08 01:36:44 INFO CodeGenerator: Code generated in 15.539975 ms
24/07/08 01:36:44 INFO CodeGenerator: Code generated in 8.813 ms


In [18]:
spark.sql("use climatedata")

24/07/08 01:38:15 INFO HiveMetaStore: 0: get_database: climatedata
24/07/08 01:38:15 INFO audit: ugi=azureuser	ip=unknown-ip-addr	cmd=get_database: climatedata	


DataFrame[]

In [19]:
tables = spark.sql("SHOW TABLES")
tables.show()

+-----------+------------+-----------+
|  namespace|   tableName|isTemporary|
+-----------+------------+-----------+
|climatedata|climate_data|      false|
+-----------+------------+-----------+



24/07/08 01:38:34 INFO HiveMetaStore: 0: get_database: climatedata
24/07/08 01:38:34 INFO audit: ugi=azureuser	ip=unknown-ip-addr	cmd=get_database: climatedata	
24/07/08 01:38:34 INFO HiveMetaStore: 0: get_database: climatedata
24/07/08 01:38:34 INFO audit: ugi=azureuser	ip=unknown-ip-addr	cmd=get_database: climatedata	
24/07/08 01:38:34 INFO HiveMetaStore: 0: get_tables: db=climatedata pat=*
24/07/08 01:38:34 INFO audit: ugi=azureuser	ip=unknown-ip-addr	cmd=get_tables: db=climatedata pat=*	
24/07/08 01:38:34 INFO CodeGenerator: Code generated in 14.727268 ms
24/07/08 01:38:34 INFO CodeGenerator: Code generated in 3.757243 ms
24/07/08 01:38:34 INFO CodeGenerator: Code generated in 3.762743 ms
